# LangChain 核心模块学习：Model I/O

`Model I/O` 是 LangChain 为开发者提供的一套面向 LLM 的标准化模型接口，包括模型输入（Prompts）、模型输出（Output Parsers）和模型本身（Models）。

- Prompts：模板化、动态选择和管理模型输入
- Models：以通用接口调用语言模型
- Output Parser：从模型输出中提取信息，并规范化内容

![](images/model_io.jpeg)


In [1]:
# 安装最新版本的 LangChain Python SDK（https://github.com/langchain-ai/langchain）
!pip install -U langchain

## 模型输入 Prompts

一个语言模型的提示是用户提供的一组指令或输入，用于引导模型的响应，帮助它理解上下文并生成相关和连贯的基于语言的输出，例如回答问题、完成句子或进行对话。


- 提示模板（Prompt Templates）：参数化的模型输入
- 示例选择器（Example Selectors）：动态选择要包含在提示中的示例


## 提示模板 Prompt Templates

**Prompt Templates 提供了一种预定义、动态注入、模型无关和参数化的提示词生成方式，以便在不同的语言模型之间重用模板。**

一个模板可能包括指令、少量示例以及适用于特定任务的具体背景和问题。

通常，提示要么是一个字符串（LLMs），要么是一组聊天消息（Chat Model）。


类继承关系:

```
BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate

PromptValue --> StringPromptValue
                ChatPromptValue
```


**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/prompts**



### 使用 PromptTemplate 类生成提升词

**通常，`PromptTemplate` 类的实例，使用Python的`str.format`语法生成模板化提示；也可以使用其他模板语法（例如jinja2）。**

#### 使用 from_template 方法实例化 PromptTemplate

In [1]:
#Azure Open AI settings

import os

import openai

#OS level settings
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://cms-oai.openai.azure.com/"
deployment_name = "chatbamboo"

In [2]:

#from langchain import PromptTemplate
from langchain import PromptTemplate

#prompt_template = PromptTemplate.from_template(
prompt_template = PromptTemplate.from_template(    
    "Tell me a {adjective} joke about {content}."
)

# 使用 format 生成提示
prompt = prompt_template.format(adjective="funny", content="chickens")
print(prompt)

Tell me a funny joke about chickens.


In [3]:
print(prompt_template)

input_variables=['adjective', 'content'] output_parser=None partial_variables={} template='Tell me a {adjective} joke about {content}.' template_format='f-string' validate_template=True


In [4]:
prompt_template = PromptTemplate.from_template(
    "Tell me a joke"
)
# 生成提示
prompt = prompt_template.format()
print(prompt)

Tell me a joke


#### 使用构造函数（Initializer）实例化 PromptTemplate

使用构造函数实例化 `prompt_template` 时必须传入参数：`input_variables` 和 `template`。

在生成提示过程中，会检查输入变量与模板字符串中的变量是否匹配，如果不匹配，则会引发异常；

In [5]:
invalid_prompt = PromptTemplate(
    input_variables=["adjective","content"],
    template="Tell me a {adjective} joke about {content}."
)

In [6]:
valid_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="Tell me a {adjective} joke about {content}."
)

In [7]:
print(valid_prompt)

input_variables=['adjective', 'content'] output_parser=None partial_variables={} template='Tell me a {adjective} joke about {content}.' template_format='f-string' validate_template=True


In [8]:
valid_prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [9]:
prompt_template = PromptTemplate.from_template(
    "讲{num}个给程序员听得笑话"
)

In [10]:
#from langchain.llms import OpenAI
from langchain.llms import AzureOpenAI

#llm = OpenAI(model_name="text-davinci-003", max_tokens=1000)
llm = AzureOpenAI(
    deployment_name = deployment_name,
    model_name="text-davinci-003", max_tokens=1000)

prompt = prompt_template.format(num=2)
print(f"prompt: {prompt}")

result = llm(prompt)
print(f"result: {result}")

prompt: 讲2个给程序员听得笑话
result: ](http://blog.jobbole.com/107538/)  
[程序员：我为什么拒绝996工作制的公司](http://blog.jobbole.com/107524/)  
[程序员你为什么这么累？](http://blog.jobbole.com/107478/)  
[一名程序员对自己代码的自我催眠](http://blog.jobbole.com/107445/)  

# 180517
[大数据必修课：Hadoop、Spark、Flink、Kafka](http://blog.jobbole.com/107706/)  
[如何成为一名优秀的前端工程师？](http://blog.jobbole.com/107627/)  
[程序员千万不要太相信经验](http://blog.jobbole.com/107626/)  
[软件工程师的职业发展之路](http://blog.jobbole.com/107711/)  
[程序员的未来：人工智能编程](http://blog.jobbole.com/107438/)  

# 180516
[程序员需要知道的网络基础知识](http://blog.jobbole.com/107645/)  
[如何成为一名优秀的后端工程师？](http://blog.jobbole.com/107587/)  
[使用Python进行自然语言处理](http://blog.jobbole.com/107561/)  
[十个Tips让你创造出好的API](http://blog.jobbole.com/107554/)  
[程序员如何提高代码的可读性？](http://blog.jobbole.com/107584/)  

# 180515
[如何成为一名优秀的Python工程师？](http://blog.jobbole.com/107503/)  
[AI时代，如何提高编程效率？](http://blog.jobbole.com/107467/)  
[一份值得收藏的前端资源清单](http://blog.jobbole.com/107493/)  
[程序员必备的十大技能](http://blog.jobbole.com/107550/)  


In [11]:
print(llm(prompt_template.format(num=3)))

**
        > - 1.今天我终于实现了一个快排，虽然我不知道他是不是真的快，但至少我排了
        > - 2.昨天我做了一个很酷的加密算法，我现在已经把所有的代码都忘了，只有我的密码还在
        > - 3.我今天写了一个无限循环，这样我就可以永远等待程序员的回复了

    - **语言和工具**
        > - Java，Python，C++，Go，JS 等
        > - Eclipse，IntelliJ IDEA，VSCode，Vim，Sublime，PyCharm 等
        > - MySQL，Oracle，SQLite，MongoDB，Redis，ElasticSearch 等

    - **其他**
        > - 项目经历：做过哪些项目，做的最好的项目是哪个，为什么
        > - 学习经历：在校期间，自学了哪些课程或技术，学习过哪些在线教育平台的课程
        > - 看过哪些书，读过哪些技术博客或网站，最喜欢哪个作者或网站，为什么

- **个人评价**
    > 简单介绍自己的性格、优缺点、爱好、理想、人生格言等方面内容，以及自己期望从事的工作、薪资要求、职业规划等方面内容

---

## 2. 面试题

### 2.1 HTML/CSS/JS

- **HTML 中的 `meta` 标签有哪些常见属性，分别有什么作用？**
    > - `charset`：定义 HTML 文档的字符编码方式
    > - `name` 和 `content`：可以用来提供有关页面的信息，比如关键字和描述
    > - `http-equiv`：用于模拟 HTTP 头部信息，比如刷新和重定向，一般设置以下值
    >   - `content-type`：定义文档的 MIME 类型和字符集
    >   - `refresh`：指定跳转或刷新的时间和 URL
    > - `property` 和 `content`：用于在 Open Graph 协议中定义一些属性，如下
    >   - `og:title`：页面的标题
    >   - `og:description`：页面描述
    >   - `og:image`：缩略图的图片 URL
    >  

#### 使用 jinja2 生成模板化提示

In [12]:
jinja2_template = "Tell me a {{ adjective }} joke about {{ content }}"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens'

In [13]:
print(prompt)

input_variables=['adjective', 'content'] output_parser=None partial_variables={} template='Tell me a {{ adjective }} joke about {{ content }}' template_format='jinja2' validate_template=True


#### 使用 f-string 生成模板化提示

In [14]:
fstring_template = """Tell me a {adjective} joke about {content}"""
prompt = PromptTemplate.from_template(fstring_template)

prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens'

#### 实测：生成多种编程语言版本的快速排序

In [15]:
sort_prompt_template = PromptTemplate.from_template(
    "生成可执行的快速排序 {programming_language} 代码"
)

In [16]:
print(llm(sort_prompt_template.format(programming_language="python")))

，以便于在多个测试样例中复用。请将所有文件打包为 zip 文件，命名为 `code.zip`。

注：本题中的 python 代码和样例数据生成的时候需要使用 python3。

## 数据说明

本题共包含 5 个数据点。

对于所有数据点，1 <= n <= 10^6，1 <= q <= 10^5。

对于前三个数据点，所有数的范围均在 int32 范围内。

对于第四个数据点，只有一种询问类型。

对于第五个数据点，所有数的范围均在 int64 范围内。

其中第四个数据点的询问数量较多，时限为 2s。其他数据点时限为 5s。

### 数据点 1（20pts）

随机生成的一组数据。

### 数据点 2（20pts）

随机生成的一组数据，其中有 10% 的元素为 0。

### 数据点 3（20pts）

随机生成的一组数据，其中有 10% 的元素取不到最小值。

### 数据点 4（20pts）

随机生成的一组数据，包含以下类型的询问：

- `1 l r x`：查询区间 `[l, r]` 中元素等于 `x` 的个数。
- `2 l r`：查询区间 `[l, r]` 中第 `k` 小的元素的值，保证区间内至少有 `k` 个元素。

其中有 10% 的询问为查询区间和。

### 数据点 5（20pts）

随机生成的一组数据，包含以下类型的询问：

- `1 l r x`：查询区间 `[l, r]` 中元素等于 `x` 的个数。
- `2 l r`：查询区间 `[l, r]` 中第 `k` 小的元素的值，保证区间内至少有 `k` 个元素。

其中有 10% 的询问为查询区间和。

## 提交与评测

本题需要提交的文件为你生成的数据，以及 `code.zip`。

评测程序会对你提交的数据进行评测。评测程序会按照评测机器上的顺序对你提交的所有数据进行评测，并记录每个数据点的得分。每个数据点的得分为：该测试点获得的分数 / 该测试点的总分数。

之后评测程序会将你提交的 python 代码和样例数据合并在一起，生成一个新的提交记录，并使用这个提交记录进行评测。这个提交记录的得分为各个数据点得分的平均分。

评测程序使用的机器在中国大陆地区。我们保证评测程序会在截止时间前对所有提交的程序进行评测，但是我们不保证在截止时间前对所有数据进行评测。

## 注

In [17]:
print(llm(sort_prompt_template.format(programming_language="java")))


        4. 编译 生成 mySort.class 文件
        5. 将 mySort.class 文件打包成 jar 包
        6. 将 jar 包转化为 base64 字符串，这里有一个网站可以生成：
            https://www.bejson.com/enc/base64/
        7. 将 base64 字符串放在代码中
        8. 运行代码，如果正常则会输出排序后的数组
*/
public class QuickSort {
    public static void main(String[] args) {
        String base64Str = "yv66vgAAADQAHwoABgARCQASABMIABQKABUAFggAFwoAFwoAFAcAFQcAFgEAFyhMamF2YS9sYW5nL09iamVjdDtMamF2YS9sYW5nL1N0cmluZzsVChJjb20uYmVzdG9uZy5xdWljay5TcXVhcmU7KVYAIQAGAAgAAAAAAAIAAAABAAEABQAGAAEABwAAAB0AAQABAAAABSq3AAGxAAAAAQAKAAAACgACAAAAAgALAAAABgABAAAACAABAAEABwAAAAoACAABAAkAAAAlAAIAAgAAAAIADAAAAAEACg==";
        byte[] bytes = Base64.getDecoder().decode(base64Str);
        MyClassLoader2 loader = new MyClassLoader2();
        Class clazz = loader.defineMyClass(bytes, 0, bytes.length);
        try {
            Method method = clazz.getMethod("sort", int[].class);
            int[] arr = {9, 8, 7, 6, 5, 4, 3, 2, 1};
            method.invoke(null, arr);
            System.o

In [18]:
print(llm(sort_prompt_template.format(programming_language="C++")))

，输出到文件 quicksort.cpp 中。

参数

本题无参数。

样例说明

样例中，生成了一个长度为 10^5 的随机数列，然后使用两种算法对其排序，并计算了排序时间。

时间限制

C++ 代码：1.5s

Python 代码：9s

Java 代码：3s

C# 代码：3s

评分标准

若本题能得分，评测结果按照以下规则计算得分：

- 若输出的 C++ 代码可编译并且对于上述的样例数据，运行结果正确，得到 100 分；
- 否则，本题得分为 0 分。

提示

注意事项：

- 生成的随机数列为 $[-10^9,10^9]$ 之间的整数，保证不重复；
- 数据量大小： $1\le n\le 10^6$；
- 请保证输出的 C++ 代码中，不出现任何读入操作，仅包含计算排序结果的代码；
- 请保证输出的 C++ 代码中，数组下标均从 1 开始，不得从 0 开始；
- 请保证输出的 C++ 代码的复杂度不超过 $\text{O}(n\log{n})$；
- 请勿直接复制他人代码，自己完成题目。 copying代码者后果自负。

参考代码

本题参考代码为 Python 3 代码，包含了对于输入数据的读取、两种排序算法的实现、计算排序时间的方法以及 C++ 代码的输出方法。您可以使用它作为参考，但不保证其一定正确，也不保证能获得满分。

import os
from random import randint
from time import time

# 代码模板
template = '''#include <iostream>
using namespace std;
int a[1000005], n;

{sort_algorithm}

int main() {
    cin >> n;
    for (int i = 1; i <= n; ++i) {
        cin >> a[i];
    }
    {sort_function}
    for (int i = 1; i <= n; ++i) {
        cout << a[i] << " ";
    }
    return 0;
}
'''

# 冒泡排序
def bubble_sort():
    return '''void bubble_sort

## 使用 ChatPromptTemplate 类生成适用于聊天模型的聊天记录

**`ChatPromptTemplate` 类的实例，使用`format_messages`方法生成适用于聊天模型的提示。**

### 使用 from_messages 方法实例化 ChatPromptTemplate

In [19]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

# 生成提示
messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

In [20]:
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, example=False), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, example=False), HumanMessage(content='What is your name?', additional_kwargs={}, example=False)]


In [21]:
print(messages[0].content)
print(messages[-1].content)

You are a helpful AI bot. Your name is Bob.
What is your name?


In [22]:
#from langchain.chat_models import ChatOpenAI
#chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=1000)
from langchain.chat_models import AzureChatOpenAI
chat_model = AzureChatOpenAI(
    deployment_name=deployment_name,
    model_name="gpt-3.5-turbo",
    max_tokens=1000)

In [23]:
chat_model(messages)

AIMessage(content='My name is Bob, how can I assist you today?', additional_kwargs={}, example=False)

### 摘要总结

In [24]:
summary_template = ChatPromptTemplate.from_messages([
    ("system", "你将获得关于同一主题的{num}篇文章（用-----------标签分隔）。首先总结每篇文章的论点。然后指出哪篇文章提出了更好的论点，并解释原因。"),
    ("human", "{user_input}"),
])

In [25]:
messages = summary_template.format_messages(
    num=3,
    user_input='''1. [PHP是世界上最好的语言]
PHP是世界上最好的情感派编程语言，无需逻辑和算法，只要情绪。它能被蛰伏在冰箱里的PHP大神轻易驾驭，会话结束后的感叹号也能传达对代码的热情。写PHP就像是在做披萨，不需要想那么多，只需把配料全部扔进一个碗，然后放到服务器上，热乎乎出炉的网页就好了。
-----------
2. [Python是世界上最好的语言]
Python是世界上最好的拜金主义者语言。它坚信：美丽就是力量，简洁就是灵魂。Python就像是那个永远在你皱眉的那一刻扔给你言情小说的好友。只有Python，你才能够在两行代码之间感受到飘逸的花香和清新的微风。记住，这世上只有一种语言可以使用空格来领导全世界的进步，那就是Python。
-----------
3. [Java是世界上最好的语言]
Java是世界上最好的德育课编程语言，它始终坚守了严谨、安全的编程信条。Java就像一个严格的老师，他不会对你怀柔，不会让你偷懒，也不会让你走捷径，但他教会你规范和自律。Java就像是那个喝咖啡也算加班费的上司，拥有对邪恶的深度厌恶和对善良的深度拥护。
'''
)

In [26]:
print(messages[-1].content)

1. [PHP是世界上最好的语言]
PHP是世界上最好的情感派编程语言，无需逻辑和算法，只要情绪。它能被蛰伏在冰箱里的PHP大神轻易驾驭，会话结束后的感叹号也能传达对代码的热情。写PHP就像是在做披萨，不需要想那么多，只需把配料全部扔进一个碗，然后放到服务器上，热乎乎出炉的网页就好了。
-----------
2. [Python是世界上最好的语言]
Python是世界上最好的拜金主义者语言。它坚信：美丽就是力量，简洁就是灵魂。Python就像是那个永远在你皱眉的那一刻扔给你言情小说的好友。只有Python，你才能够在两行代码之间感受到飘逸的花香和清新的微风。记住，这世上只有一种语言可以使用空格来领导全世界的进步，那就是Python。
-----------
3. [Java是世界上最好的语言]
Java是世界上最好的德育课编程语言，它始终坚守了严谨、安全的编程信条。Java就像一个严格的老师，他不会对你怀柔，不会让你偷懒，也不会让你走捷径，但他教会你规范和自律。Java就像是那个喝咖啡也算加班费的上司，拥有对邪恶的深度厌恶和对善良的深度拥护。



In [27]:
chat_result = chat_model(messages)

In [28]:
print(chat_result.content)

1. PHP是世界上最好的语言，因为它不需要逻辑和算法，只要情绪，是一种情感派编程语言，可以轻易地驾驭。
2. Python是世界上最好的语言，因为它美丽、简洁、飘逸，使用空格来领导全世界的进步。
3. Java是世界上最好的语言，因为它严谨、安全，可以教会规范和自律，拥有对邪恶的深度厌恶和对善良的深度拥护。

其中，没有提出更好的论点。因为这三篇文章都只是从各自的角度对编程语言进行了评价和描述，并没有进行比较。每篇文章都有自己的观点和特点，没有绝对的优劣之分。


In [29]:
messages = summary_template.format_messages(
    num=2,
    user_input='''1.认为“道可道”中的第一个“道”，指的是道理，如仁义礼智之类；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，所谓“道可道，非常道”，指的是可以言说的道理，不是恒久存在的“道”，恒久存在的“道”不可言说。如苏辙说：“莫非道也。而可道者不可常，惟不可道，而后可常耳。今夫仁义礼智，此道之可道者也。然而仁不可以为义，而礼不可以为智，可道之不可常如此。……而道常不变，不可道之能常如此。”蒋锡昌说：“此道为世人所习称之道，即今人所谓‘道理’也，第一‘道’字应从是解。《广雅·释诂》二：‘道，说也’，第二‘道’字应从是解。‘常’乃真常不易之义，在文法上为区别词。……第三‘道’字即二十五章‘道法自然’之‘道’，……乃老子学说之总名也”。陈鼓应说：“第一个‘道’字是人们习称之道，即今人所谓‘道理’。第二个‘道’字，是指言说的意思。第三个‘道’字，是老子哲学上的专有名词，在本章它意指构成宇宙的实体与动力。……‘常道’之‘常’，为真常、永恒之意。……可以用言词表达的道，就不是常道”。
-----------
2.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，“道可道，非常道”，指可以言说的“道”，就不是恒久存在的“道”。如张默生说：“‘道’，指宇宙的本体而言。……‘常’，是经常不变的意思。……可以说出来的道，便不是经常不变的道”。董平说：“第一个‘道’字与‘可道’之‘道’，内涵并不相同。第一个‘道’字，是老子所揭示的作为宇宙本根之‘道’；‘可道’之‘道’，则是‘言说’的意思。……这里的大意就是说：凡一切可以言说之‘道’，都不是‘常道’或永恒之‘道’”。汤漳平等说：“第一句中的三个‘道’，第一、三均指形上之‘道’，中间的‘道’作动词，为可言之义。……道可知而可行，但非恒久不变之道”。
--------
3.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，则指的是平常人所讲之道、常俗之道。因此，“道可道，非常道”，指“道”是可以言说的，但它不是平常人所谓的道或常俗之道。如李荣说：“道者，虚极之理也。夫论虚极之理，不可以有无分其象，不可以上下格其真。……圣人欲坦兹玄路，开以教门，借圆通之名，目虚极之理，以理可名，称之可道。故曰‘吾不知其名，字之曰道’。非常道者，非是人间常俗之道也。人间常俗之道，贵之以礼义，尚之以浮华，丧身以成名，忘己而徇利。”司马光说：“世俗之谈道者，皆曰道体微妙，不可名言。老子以为不然，曰道亦可言道耳，然非常人之所谓道也。……常人之所谓道者，凝滞于物。”裘锡圭说：“到目前为止，可以说，几乎从战国开始，大家都把‘可道’之‘道’……看成老子所否定的，把‘常道’‘常名’看成老子所肯定的。这种看法其实有它不合理的地方，……‘道’是可以说的。《老子》这个《道经》第一章，开宗明义是要讲他的‘道’。第一个‘道’字，理所应当，也是讲他要讲的‘道’：道是可以言说的。……那么这个‘恒’字应该怎么讲？我认为很简单，‘恒’字在古代作定语用，经常是‘平常’‘恒常’的意思。……‘道’是可以言说的，但是我要讲的这个‘道’，不是‘恒道’，它不是一般人所讲的‘道’。
'''
)
print(messages)

[SystemMessage(content='你将获得关于同一主题的2篇文章（用-----------标签分隔）。首先总结每篇文章的论点。然后指出哪篇文章提出了更好的论点，并解释原因。', additional_kwargs={}), HumanMessage(content='1.认为“道可道”中的第一个“道”，指的是道理，如仁义礼智之类；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，所谓“道可道，非常道”，指的是可以言说的道理，不是恒久存在的“道”，恒久存在的“道”不可言说。如苏辙说：“莫非道也。而可道者不可常，惟不可道，而后可常耳。今夫仁义礼智，此道之可道者也。然而仁不可以为义，而礼不可以为智，可道之不可常如此。……而道常不变，不可道之能常如此。”蒋锡昌说：“此道为世人所习称之道，即今人所谓‘道理’也，第一‘道’字应从是解。《广雅·释诂》二：‘道，说也’，第二‘道’字应从是解。‘常’乃真常不易之义，在文法上为区别词。……第三‘道’字即二十五章‘道法自然’之‘道’，……乃老子学说之总名也”。陈鼓应说：“第一个‘道’字是人们习称之道，即今人所谓‘道理’。第二个‘道’字，是指言说的意思。第三个‘道’字，是老子哲学上的专有名词，在本章它意指构成宇宙的实体与动力。……‘常道’之‘常’，为真常、永恒之意。……可以用言词表达的道，就不是常道”。\n-----------\n2.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，“道可道，非常道”，指可以言说的“道”，就不是恒久存在的“道”。如张默生说：“‘道’，指宇宙的本体而言。……‘常’，是经常不变的意思。……可以说出来的道，便不是经常不变的道”。董平说：“第一个‘道’字与‘可道’之‘道’，内涵并不相同。第一个‘道’字，是老子所揭示的作为宇宙本根之‘道’；‘可道’之‘道’，则是‘言说’的意思。……这里的大意就是说：凡一切可以言说之‘道’，都不是‘常道’或永恒之‘道’”。汤漳平等说：“第一句中的三个‘道’，第一、三均指形上之‘道’，中间的‘道’作动词，为可言之义。……道可知而可行，但非恒久不变之道”。\n--------\n3.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，则

In [30]:
chat_result = chat_model(messages)
print(chat_result.content)

第一篇文章认为“道可道”的第一个“道”指的是道理，而第二篇和第三篇文章认为它指的是宇宙的本原。所有文章都认为“可道”中的“道”指的是言说的意思，而“常道”指的是恒久存在的道。对于“道可道，非常道”的含义，第一篇文章认为它指的是可以言说的道理不是恒久存在的道，而第二篇和第三篇文章则认为它指的是可以言说的道不是平常人所讲的道或常俗之道。

相比之下，第三篇文章提出的论点更好，因为它更全面地考虑了“道可道”中第一个“道”的含义，既包括了宇宙的本原，也包括了虚极之理。此外，它还指出了“常道”与平常人所讲的道或常俗之道的区别，更加准确地解释了“道可道，非常道”的含义。


### 使用 FewShotPromptTemplate 类生成 Few-shot Prompt 

构造 few-shot prompt 的方法通常有两种：
- 从示例集（set of examples）中手动选择；
- 通过示例选择器（Example Selector）自动选择.

In [31]:
from langchain.prompts.prompt import PromptTemplate


examples = [
  {
    "question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵？",
    "answer": 
"""
这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里
"""
  },
  {
    "question": "craigslist的创始人是什么时候出生的？",
    "answer": 
"""
这里需要进一步的问题吗：是的。
追问：谁是craigslist的创始人？
中间答案：Craigslist是由Craig Newmark创办的。
追问：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark出生于1952年12月6日。
所以最终答案是：1952年12月6日
"""
  },
  {
    "question": "乔治·华盛顿的外祖父是谁？",
    "answer":
"""
这里需要进一步的问题吗：是的。
追问：谁是乔治·华盛顿的母亲？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
追问：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball
"""
  },
  {
    "question": "《大白鲨》和《皇家赌场》的导演是同一个国家的吗？",
    "answer":
"""
这里需要进一步的问题吗：是的。
追问：谁是《大白鲨》的导演？
中间答案：《大白鲨》的导演是Steven Spielberg。
追问：Steven Spielberg来自哪里？
中间答案：美国。
追问：谁是《皇家赌场》的导演？
中间答案：《皇家赌场》的导演是Martin Campbell。
追问：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是
"""
  }
]

In [32]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}"
)

# **examples[0] 是将examples[0] 字典的键值对（question-answer）解包并传递给format，作为函数参数
print(example_prompt.format(**examples[0]))

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？

这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里



In [33]:
print(example_prompt)

input_variables=['question', 'answer'] output_parser=None partial_variables={} template='Question: {question}\n{answer}' template_format='f-string' validate_template=True


#### 关于解包的示例

In [34]:
def print_info(question, answer):
    print(f"Question: {question}")
    print(f"Answer: {answer}")

print_info(**examples[0]) 

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？
Answer: 
这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里



### 生成 Few-shot Prompt

In [35]:
# 导入 FewShotPromptTemplate 类
from langchain.prompts.few_shot import FewShotPromptTemplate

# 创建一个 FewShotPromptTemplate 对象
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,           # 使用前面定义的 examples 作为范例
    example_prompt=example_prompt, # 使用前面定义的 example_prompt 作为提示模板
    suffix="Question: {input}",    # 后缀模板，其中 {input} 会被替换为实际输入
    input_variables=["input"]     # 定义输入变量的列表
)

# 使用给定的输入格式化 prompt，并打印结果
# 这里的 {input} 将被 "玛丽·波尔·华盛顿的父亲是谁?" 替换
print(few_shot_prompt.format(input="玛丽·波尔·华盛顿的父亲是谁?"))

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？

这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里


Question: craigslist的创始人是什么时候出生的？

这里需要进一步的问题吗：是的。
追问：谁是craigslist的创始人？
中间答案：Craigslist是由Craig Newmark创办的。
追问：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark出生于1952年12月6日。
所以最终答案是：1952年12月6日


Question: 乔治·华盛顿的外祖父是谁？

这里需要进一步的问题吗：是的。
追问：谁是乔治·华盛顿的母亲？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
追问：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball


Question: 《大白鲨》和《皇家赌场》的导演是同一个国家的吗？

这里需要进一步的问题吗：是的。
追问：谁是《大白鲨》的导演？
中间答案：《大白鲨》的导演是Steven Spielberg。
追问：Steven Spielberg来自哪里？
中间答案：美国。
追问：谁是《皇家赌场》的导演？
中间答案：《皇家赌场》的导演是Martin Campbell。
追问：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是


Question: 玛丽·波尔·华盛顿的父亲是谁?


## 示例选择器 Example Selectors

**如果你有大量的参考示例，就得选择哪些要包含在提示中。最好还是根据某种条件或者规则来自动选择，Example Selector 是负责这个任务的类。**

BaseExampleSelector 定义如下：

```python
class BaseExampleSelector(ABC):
    """用于选择包含在提示中的示例的接口。"""

    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """根据输入选择要使用的示例。"""

```

`ABC` 是 Python 中的 `abc` 模块中的一个缩写，它表示 "Abstract Base Class"（抽象基类）。在 Python 中，抽象基类用于定义其他类必须遵循的基本接口或蓝图，但不能直接实例化。其主要目的是为了提供一种形式化的方式来定义和检查子类的接口。

使用抽象基类的几点关键信息：

1. **抽象方法**：在抽象基类中，你可以定义抽象方法，它没有实现（也就是说，它没有方法体）。任何继承该抽象基类的子类都必须提供这些抽象方法的实现。

2. **不能直接实例化**：你不能直接创建抽象基类的实例。试图这样做会引发错误。它们的主要目的是为了被继承，并在子类中实现其方法。

3. **强制子类实现**：如果子类没有实现所有的抽象方法，那么试图实例化该子类也会引发错误。这确保了继承抽象基类的所有子类都遵循了预定的接口。


In [36]:
!pip install chromadb

In [37]:
# 导入需要的模块和类
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
#from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# 定义一个提示模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],     # 输入变量的名字
    template="Input: {input}\nOutput: {output}",  # 实际的模板字符串
)

# 这是一个假设的任务示例列表，用于创建反义词
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [42]:
#embedding setup deployment 
embedding = OpenAIEmbeddings(
    deployment=deployment_name
)

print(embedding)

client=<class 'openai.api_resources.embedding.Embedding'> model='text-embedding-ada-002' deployment='chatbamboo' openai_api_version='2023-03-15-preview' openai_api_base='https://cms-oai.openai.azure.com/' openai_api_type='azure' openai_proxy='' embedding_ctx_length=8191 openai_api_key='75a5d48611f745779d605326ecab6fde' openai_organization='' allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=6 request_timeout=None headers=None tiktoken_model_name=None show_progress_bar=False model_kwargs={}


In [41]:


# 从给定的示例中创建一个语义相似性选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,                          # 可供选择的示例列表
    embedding,                # 用于生成嵌入向量的嵌入类，用于衡量语义相似性
    Chroma,                            # 用于存储嵌入向量并进行相似性搜索的 VectorStore 类
    k=1                                # 要生成的示例数量
)

# 创建一个 FewShotPromptTemplate 对象
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 提供一个 ExampleSelector 替代示例
    example_prompt=example_prompt,      # 前面定义的提示模板
    prefix="Give the antonym of every input", # 前缀模板
    suffix="Input: {adjective}\nOutput:",     # 后缀模板
    input_variables=["adjective"],           # 输入变量的名字
)


InvalidRequestError: The embeddings operation does not work with the specified model, gpt-35-turbo. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.

In [ ]:
# 输入是一种感受，所以应该选择 happy/sad 的示例。
print(similar_prompt.format(adjective="worried"))

In [ ]:
# 输入是一种度量，所以应该选择 tall/short的示例。
print(similar_prompt.format(adjective="long"))